In [1]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install re
!pip install nltk
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 30.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 8.5 MB/s eta 0:00:000:00:01
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 7.1 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 9.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.7 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 9.3 MB/s eta 0:00:00eta 0:00:01
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import openpyxl

### Load datasets

#### Lolland-Falster

In [31]:
lf_df = pd.read_excel('../../../DATA/NEWSPAPERS/Maribo_final_docs.xlsx')

In [32]:
lf_df['category'] = lf_df['seneste_rubrik'].str.replace(r'\.?_x000D_', '', regex=True)
lf_df['article_length'] = lf_df['text'].apply(lambda x: len(str(x).split()))
lf_df['characters'] = lf_df['text'].str.len()
lf_df['date'] = pd.to_datetime(lf_df['nøgle'].str[:10], errors='coerce')

In [33]:
lf_df['clean_category'] = lf_df['category']

replacements = {'(Jndsendt.)': 'Jndenlandsk', # they get mixed up in the classification 
                'Jndsendt': 'Jndenlandsk', #they get mixed up in the classification
                'Bekiendtgiørelser': 'Bekjendtgjørelser',
                'Bekiendtgjørelser': 'Bekjendtgjørelser',
                'Bekjendtgjörelser': 'Bekjendtgjørelser',
                'Udenlandske Efterretninger': 'Udenlandsk',
                'Udenlandske Nyheder': 'Udenlandsk',
                'Fædrelandet': 'Jndenlandsk', # these get renamed after 1836
                'Blandet': 'Udenlandsk'} # these get renamed after 1836

lf_df['clean_category'] = lf_df['clean_category'].replace(replacements)

In [34]:
lf_df['article_id'] = [f'lol_{i+1:06d}' for i in range(len(lf_df))]
lf_df = lf_df[['article_id', 'nøgle', 'date', 'text', 'category', 'clean_category', 'article_length', 'characters']]
lf_df.head()

,article_id,nøgle,date,text,category,clean_category,article_length,characters
0,lol_000001,1809-03-07_5,1809-03-07,Jndenlandsk. Helsingøer den 26de Februar. J Fo...,Jndenlandsk,Jndenlandsk,298,1592
1,lol_000002,1809-03-07_8,1809-03-07,Kjøbenhavn den 27 Februarii. Kornpriserne i Kb...,Jndenlandsk,Jndenlandsk,81,513
2,lol_000003,1809-03-07_9,1809-03-07,De danske Officerer af Linieskibet Prinds Chri...,Jndenlandsk,Jndenlandsk,31,205
3,lol_000004,1809-03-07_10,1809-03-07,Caffe koster nu paa det Vestindiske Compagnie ...,Jndenlandsk,Jndenlandsk,48,257
4,lol_000005,1809-03-07_11,1809-03-07,"For kort Tid siden bleve de, der boe ved Veste...",Jndenlandsk,Jndenlandsk,80,445


In [50]:
lf_df.to_csv('../../../DATA/NEWSPAPERS/ready_for_embs/lf_all.csv')

#### Aalborg (1818-1827)

In [35]:
aal_df1 = pd.read_json('../../../DATA/NEWSPAPERS/allborg-shiny-app/2024-09-20T145611.200.json')
aal_df1 = aal_df1['data'].apply(pd.Series)
aal_df1.columns = ['nøgle', 'text', 'category', 'date', 'Tags', 'IDs']
aal_df1 = aal_df1[['nøgle', 'text', 'category', 'date']]
aal_df1['date'] = pd.to_datetime(aal_df1['date'], errors='coerce')

aal_df1.head()

,nøgle,text,category,date
0,2,J den engelske Captain Sir James Lucas Yeos Ef...,Afrika.,1818-01-29
1,4,Den Hamborgske flyvende Major von Glöden har f...,Blandede Efterretninger.,1818-01-29
2,5,"De Viinske Aviser tale meget vidt og bredt om,...",Blandede Efterretninger.,1818-01-29
3,6,En Hamborger Handelsmand har havt den sieldne ...,Blandede Efterretninger.,1818-01-29
4,7,Det haarde Veir vi her havde Natten til den 16...,Blandede Efterretninger.,1818-01-29


#### Aalborg (1827-1842)

In [36]:
aal_df2 = pd.read_excel('../../../DATA/NEWSPAPERS/Aalborg_1827_1842.xlsx')
aal_df2['category'] = aal_df2['seneste_rubrik'].str.replace(r'\.?_x000D_', '', regex=True)
aal_df2['date'] = pd.to_datetime(aal_df2['nøgle'].str[:10], errors='coerce')
aal_df2 = aal_df2[['nøgle', 'text', 'category', 'date']]
aal_df2.head()

,nøgle,text,category,date
0,1827-04-02_1,Blandinger. Værdien af de fra Landbankerne i E...,Blandinger,1827-04-02
1,1827-04-02_3,Kongen af England har overdraget Marquess af H...,Blandinger,1827-04-02
2,1827-04-02_4,Captain Parry agtede med Skibet Hecla den 25de...,Blandinger,1827-04-02
3,1827-04-02_5,J Spaniens Hovedstad finde paa den sidste Tid ...,Blandinger,1827-04-02
4,1827-04-02_6,"Den Cadizer Handelstidende Dos Mundos melder, ...",Blandinger,1827-04-02


In [37]:
aal_df = pd.concat([aal_df1, aal_df2], ignore_index=True)

In [38]:
aal_df['article_id'] = [f'aal_{i+1:06d}' for i in range(len(aal_df))]
aal_df['article_length'] = aal_df['text'].apply(lambda x: len(str(x).split()))
aal_df['characters'] = aal_df['text'].str.len()
aal_df['clean_category'] = '-1'
aal_df = aal_df[['article_id', 'nøgle', 'date', 'text', 'category', 'clean_category', 'article_length', 'characters']]
aal_df.head()

,article_id,nøgle,date,text,category,clean_category,article_length,characters
0,aal_000001,2,1818-01-29,J den engelske Captain Sir James Lucas Yeos Ef...,Afrika.,-1,301,1867
1,aal_000002,4,1818-01-29,Den Hamborgske flyvende Major von Glöden har f...,Blandede Efterretninger.,-1,118,720
2,aal_000003,5,1818-01-29,"De Viinske Aviser tale meget vidt og bredt om,...",Blandede Efterretninger.,-1,20,95
3,aal_000004,6,1818-01-29,En Hamborger Handelsmand har havt den sieldne ...,Blandede Efterretninger.,-1,45,269
4,aal_000005,7,1818-01-29,Det haarde Veir vi her havde Natten til den 16...,Blandede Efterretninger.,-1,84,435


In [51]:
aal_df.to_csv('../../../DATA/NEWSPAPERS/ready_for_embs/aal_all.csv')

### Odense (1809-1848)

In [41]:
od_df = pd.read_excel('../../../DATA/NEWSPAPERS/Odense_1809_1848.xlsx')
od_df['category'] = od_df['seneste_rubrik'].str.replace(r'\.?_x000D_', '', regex=True)
od_df['date'] = pd.to_datetime(od_df['nøgle'].str[:10], errors='coerce')
od_df = od_df[['nøgle', 'text', 'category', 'date']]
od_df.head()

,nøgle,text,category,date
0,1809-01-02_128257,Tilbageblik paa Aaret 1808. Det 8de Aar af det...,Første rubrik,1809-01-02
1,1809-01-02_128262,"Den 16de Bulletin fra Armeen i Spanien, datere...",Første rubrik,1809-01-02
2,1809-01-02_128263,Den ved Cadix stationerede Franske Flaades Bor...,Første rubrik,1809-01-02
3,1809-01-02_128264,Snart kommer Raden nu igjen til Lissabon. J Pa...,Første rubrik,1809-01-02
4,1809-01-02_128265,Med alt dette glimter dog endnu et Fredshaab i...,Første rubrik,1809-01-02


In [42]:
od_df['article_id'] = [f'ode_{i+1:06d}' for i in range(len(od_df))]
od_df['article_length'] = od_df['text'].apply(lambda x: len(str(x).split()))
od_df['characters'] = od_df['text'].str.len()
od_df['clean_category'] = '-1'
od_df = od_df[['article_id', 'nøgle', 'date', 'text', 'category', 'clean_category', 'article_length', 'characters']]
od_df.head()

,article_id,nøgle,date,text,category,clean_category,article_length,characters
0,ode_000001,1809-01-02_128257,1809-01-02,Tilbageblik paa Aaret 1808. Det 8de Aar af det...,Første rubrik,-1,1189,7698
1,ode_000002,1809-01-02_128262,1809-01-02,"Den 16de Bulletin fra Armeen i Spanien, datere...",Første rubrik,-1,292,1785
2,ode_000003,1809-01-02_128263,1809-01-02,Den ved Cadix stationerede Franske Flaades Bor...,Første rubrik,-1,33,199
3,ode_000004,1809-01-02_128264,1809-01-02,Snart kommer Raden nu igjen til Lissabon. J Pa...,Første rubrik,-1,72,446
4,ode_000005,1809-01-02_128265,1809-01-02,Med alt dette glimter dog endnu et Fredshaab i...,Første rubrik,-1,316,1991


In [52]:
od_df.to_csv('../../../DATA/NEWSPAPERS/ready_for_embs/od_all.csv')

In [3]:
od_df = pd.read_csv('../../../DATA/NEWSPAPERS/ready_for_embs/od_all.csv')
od_df.shape

(202547, 9)

In [5]:
od_df[~od_df['date'].isnull()].to_csv('../../../DATA/NEWSPAPERS/ready_for_embs/od_all_clean.csv')

### Thisted (1824-1848)

In [43]:
thi_df = pd.read_excel('../../../DATA/NEWSPAPERS/Thisted_1824_1848.xlsx')
thi_df['category'] = thi_df['seneste_rubrik'].str.replace(r'\.?_x000D_', '', regex=True)
thi_df['date'] = pd.to_datetime(thi_df['nøgle'].str[:10], errors='coerce')
thi_df = thi_df[['nøgle', 'text', 'category', 'date']]
thi_df.head()

,nøgle,text,category,date
0,1824-05-06_1,Ved Jyllands nordlige Kyster fremtræder herved...,Første rubrik,1824-05-06
1,1824-05-06_2,Fuldkommen føler Udgiveren sit vanskelige Hver...,Første rubrik,1824-05-06
2,1824-05-06_4,Den litteraire kundskabsrige Mand beder jeg de...,Første rubrik,1824-05-06
3,1824-05-06_5,Den store engelske Taler Phillips Carakteer-Sk...,Første rubrik,1824-05-06
4,1824-05-06_6,"Stor, skummel, og egen sad han paa Thronen, li...",Første rubrik,1824-05-06


In [44]:
thi_df['article_id'] = [f'thi_{i+1:06d}' for i in range(len(thi_df))]
thi_df['article_length'] = thi_df['text'].apply(lambda x: len(str(x).split()))
thi_df['characters'] = thi_df['text'].str.len()
thi_df['clean_category'] = '-1'
thi_df = thi_df[['article_id', 'nøgle', 'date', 'text', 'category', 'clean_category', 'article_length', 'characters']]
thi_df.head()

,article_id,nøgle,date,text,category,clean_category,article_length,characters
0,thi_000001,1824-05-06_1,1824-05-06,Ved Jyllands nordlige Kyster fremtræder herved...,Første rubrik,-1,35,231
1,thi_000002,1824-05-06_2,1824-05-06,Fuldkommen føler Udgiveren sit vanskelige Hver...,Første rubrik,-1,87,545
2,thi_000003,1824-05-06_4,1824-05-06,Den litteraire kundskabsrige Mand beder jeg de...,Første rubrik,-1,55,334
3,thi_000004,1824-05-06_5,1824-05-06,Den store engelske Taler Phillips Carakteer-Sk...,Første rubrik,-1,46,309
4,thi_000005,1824-05-06_6,1824-05-06,"Stor, skummel, og egen sad han paa Thronen, li...",Første rubrik,-1,19,114


In [53]:
thi_df.to_csv('../../../DATA/NEWSPAPERS/ready_for_embs/thi_all.csv')

### Viborg (1809-1842)

In [45]:
vib_df = pd.read_excel('../../../DATA/NEWSPAPERS/Viborg_1809_1842.xlsx')
vib_df['category'] = vib_df['seneste_rubrik'].str.replace(r'\.?_x000D_', '', regex=True)
vib_df['date'] = pd.to_datetime(vib_df['nøgle'].str[:10], errors='coerce')
vib_df = vib_df[['nøgle', 'text', 'category', 'date']]
vib_df.head()

,nøgle,text,category,date
0,1809-01-01_110895,"Kaperbaaden det gode Haab, udrustet af Strandi...",Første rubrik,1809-01-01
1,1809-01-01_110896,"Kaperfører Jens Bødker har, med Koperbaaden Na...",Første rubrik,1809-01-01
2,1809-01-01_110897,J Aalborg have nogle skadefroe Mennesker til f...,Første rubrik,1809-01-01
3,1809-01-01_110898,Aarhuus. En Kudsk fra en Herregaard kom efvigt...,Første rubrik,1809-01-01
4,1809-01-01_110899,Helsingøer d. 25de Dec. Jgaar kom 7 Kiøbmandsk...,Første rubrik,1809-01-01


In [47]:
vib_df['article_id'] = [f'vib_{i+1:06d}' for i in range(len(vib_df))]
vib_df['article_length'] = vib_df['text'].apply(lambda x: len(str(x).split()))
vib_df['characters'] = vib_df['text'].str.len()
vib_df['clean_category'] = '-1'
vib_df = vib_df[['article_id', 'nøgle', 'date', 'text', 'category', 'clean_category', 'article_length', 'characters']]
vib_df.head()

,article_id,nøgle,date,text,category,clean_category,article_length,characters
0,vib_000001,1809-01-01_110895,1809-01-01,"Kaperbaaden det gode Haab, udrustet af Strandi...",Første rubrik,-1,151,960
1,vib_000002,1809-01-01_110896,1809-01-01,"Kaperfører Jens Bødker har, med Koperbaaden Na...",Første rubrik,-1,78,399
2,vib_000003,1809-01-01_110897,1809-01-01,J Aalborg have nogle skadefroe Mennesker til f...,Første rubrik,-1,112,760
3,vib_000004,1809-01-01_110898,1809-01-01,Aarhuus. En Kudsk fra en Herregaard kom efvigt...,Første rubrik,-1,146,812
4,vib_000005,1809-01-01_110899,1809-01-01,Helsingøer d. 25de Dec. Jgaar kom 7 Kiøbmandsk...,Første rubrik,-1,386,2202


In [54]:
vib_df.to_csv('../../../DATA/NEWSPAPERS/ready_for_embs/vib_all.csv')

### Aarhus

In [ ]:
aar_df = pd.read_excel('../../../ROOT/DATA/NEWSPAPERS/newspapers_excels/Aarhus_final_docs.xlsx')
aar_df['category'] = aar_df['seneste_rubrik'].str.replace(r'\.?_x000D_', '', regex=True)
aar_df['date'] = pd.to_datetime(aar_df['nøgle'].str[:10], errors='coerce')
aar_df = aar_df[['nøgle', 'text', 'category', 'date']]
aar_df.head()

,nøgle,ID,text,V1,V2,seneste_rubrik
0,1794-01-03_1,aarhusstiftstidende-1794-01-03-01-0003B-presen...,"Jndenlandske Nyeheder. Aarhuus, den 3die Janua...",2.179595,1.614622,Første rubrik
1,1794-01-03_2,aarhusstiftstidende-1794-01-03-01-0003B-presen...,"Skipper Sode her af Byen, som fører Vintermakk...",-1.155715,6.008668,Første rubrik
2,1794-01-03_3,aarhusstiftstidende-1794-01-03-01-0003B-presen...,"Bourdeaux den 9 Dec. Skipper Henrich Krull, so...",3.219819,1.759900,Første rubrik
3,1794-01-03_4,aarhusstiftstidende-1794-01-03-01-0003B-presen...,"Høieste Rets Procurator Gottlob Smith, contra ...",2.264676,0.469230,Første rubrik
4,1794-01-03_5,aarhusstiftstidende-1794-01-03-01-0003B-presen...,"Udi Sagens 1ste Post, bør Kammerherre Schinke ...",3.402384,1.387451,Første rubrik


In [7]:
aar_df['article_id'] = [f'aar_{i+1:06d}' for i in range(len(aar_df))]
aar_df['article_length'] = aar_df['text'].apply(lambda x: len(str(x).split()))
aar_df['characters'] = aar_df['text'].str.len()
aar_df['clean_category'] = '-1'
aar_df = aar_df[['article_id', 'nøgle', 'date', 'text', 'category', 'clean_category', 'article_length', 'characters']]
aar_df.head()

,article_id,nøgle,date,text,category,clean_category,article_length,characters
0,aar_000001,1794-01-03_1,1794-01-03,"Jndenlandske Nyeheder. Aarhuus, den 3die Janua...",Første rubrik,-1,6,50
1,aar_000002,1794-01-03_2,1794-01-03,"Skipper Sode her af Byen, som fører Vintermakk...",Første rubrik,-1,23,148
2,aar_000003,1794-01-03_3,1794-01-03,"Bourdeaux den 9 Dec. Skipper Henrich Krull, so...",Første rubrik,-1,415,2438
3,aar_000004,1794-01-03_4,1794-01-03,"Høieste Rets Procurator Gottlob Smith, contra ...",Første rubrik,-1,11,95
4,aar_000005,1794-01-03_5,1794-01-03,"Udi Sagens 1ste Post, bør Kammerherre Schinke ...",Første rubrik,-1,769,4087


In [8]:
aar_df.shape

(182200, 8)

In [11]:
aar_df.to_csv('../../../ROOT/DATA/NEWSPAPERS/ready_for_embs/aar_all.csv')

### Merge all articles in one file

In [55]:
all_articles = pd.concat([lf_df, aal_df, od_df, thi_df, vib_df], ignore_index=True)
alll_articles = all_articles['clean_category'].astype(str)
all_articles.shape

(686561, 8)

In [56]:
all_articles.to_csv('../../../DATA/NEWSPAPERS/ready_for_embs/all_articles.csv')

In [3]:
all_articles = pd.read_csv('../../../DATA/NEWSPAPERS/ready_for_embs/all_articles.csv', index_col=0)
all_articles.head()

/tmp/ipykernel_346/4219108675.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_articles = pd.read_csv('../../../DATA/NEWSPAPERS/ready_for_embs/all_articles.csv', index_col=0)


,article_id,nøgle,date,text,category,clean_category,article_length,characters
0,lol_000001,1809-03-07_5,1809-03-07,Jndenlandsk. Helsingøer den 26de Februar. J Fo...,Jndenlandsk,Jndenlandsk,298,1592
1,lol_000002,1809-03-07_8,1809-03-07,Kjøbenhavn den 27 Februarii. Kornpriserne i Kb...,Jndenlandsk,Jndenlandsk,81,513
2,lol_000003,1809-03-07_9,1809-03-07,De danske Officerer af Linieskibet Prinds Chri...,Jndenlandsk,Jndenlandsk,31,205
3,lol_000004,1809-03-07_10,1809-03-07,Caffe koster nu paa det Vestindiske Compagnie ...,Jndenlandsk,Jndenlandsk,48,257
4,lol_000005,1809-03-07_11,1809-03-07,"For kort Tid siden bleve de, der boe ved Veste...",Jndenlandsk,Jndenlandsk,80,445


In [7]:
all_articles[~all_articles['date'].isnull()].to_csv('../../../DATA/NEWSPAPERS/ready_for_embs/all_articles_clean.csv')

### Create subset for training classifier

In [30]:
# 1. Sample 1000 rows per category from lol_df
sampled_lol_df = (
    lf_df.groupby('clean_category', group_keys=False)
          .apply(lambda grp: grp.sample(n=1000, random_state=42))
          .reset_index(drop=True)
)[['text', 'article_id', 'clean_category']]

# 2. Sample 3000 rows from aal_df and add a new 'clean_category' column with -1
sampled_aal_df = aal_df.sample(n=3000, random_state=42).reset_index(drop=True)
sampled_aal_df['clean_category'] = -1
sampled_aal_df = sampled_aal_df[['text', 'article_id', 'clean_category']]

# 3. Concatenate the two DataFrames
new_df = pd.concat([sampled_lol_df, sampled_aal_df], ignore_index=True)

new_df.head()

/tmp/ipykernel_484/438460016.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda grp: grp.sample(n=1000, random_state=42))


,text,article_id,clean_category
0,Bekjendtgjørelser. Overfarten fra Rødbye Femme...,lol_60633,Bekjendtgjørelser
1,Endeel fine og andre ternede Tørklæder med fle...,lol_12841,Bekjendtgjørelser
2,Undvigte. Da Reserven Christen Madsen Saltvig ...,lol_08919,Bekjendtgjørelser
3,"Dette bekjendtgjøres herved, saavel til Liebha...",lol_52440,Bekjendtgjørelser
4,"Funden. Af en Soldat her paa Byens Gade, er fu...",lol_06887,Bekjendtgjørelser


In [23]:
new_df['clean_category'].value_counts()

clean_category
-1                   3000
Bekjendtgjørelser    1000
Jndenlandsk          1000
Udenlandsk           1000
Name: count, dtype: int64

In [31]:
new_df.to_csv('../embeddings/input_classifier.csv')